In [31]:
import pandas as pd
import numpy as np
dataframe=pd.read_csv('result/nid/location_dectection_seul.csv')
dataframe=dataframe.drop_duplicates(subset=['time','id'],keep='first')
dataframe=dataframe.drop_duplicates(subset=['time','center_x','center_y'],keep='first')
dataframe.head(20)

,time,id,x,y,w,h,center_x,center_y
0,4,0,495,1037,25,15,507,1044
1,4,1,842,329,23,21,853,339
2,4,2,987,209,22,30,998,224
3,5,0,496,1037,23,15,507,1044
4,5,1,843,331,20,18,853,340
5,6,0,496,1038,22,13,507,1044
6,6,1,843,331,21,19,853,340
7,7,1,843,331,21,19,853,340
8,8,1,844,332,20,18,854,341
9,11,3,994,226,13,11,1000,231


In [19]:
min_time=min(dataframe['time'])
max_time=max(dataframe['time'])

In [32]:
#init_time=min(dataframe[dataframe['id']==id]['time'])
def find_fourmis(dataframe,min_time,max_time,id,time):
    init_time=time
    #print('init_time=',init_time)
    dataframe_temp=dataframe[(dataframe['time']<time+15) & (dataframe['time']>=time)].drop_duplicates(subset=['id'],keep='last')
    x,y,time=dataframe_temp[dataframe_temp['id']==id][['center_x','center_y','time']].values[0]
    pos=np.array([x,y])
    #print('pos=',pos,'time=',time)
    if time>max_time-15:
        return dataframe
    if time>init_time:
        #print('found the same id')
        find_fourmis(dataframe,min_time,max_time,id,time)
    else: # if not, find the probable candidate
        #print('not found the same id')
        for i in range (len(dataframe_temp)):
            temp_x,temp_y,temp_time=dataframe_temp.iloc[i][['center_x','center_y','time']].values
            temp_pos=np.array([temp_x,temp_y])
            if np.linalg.norm(temp_pos-pos)<(temp_time-time)*3:
                id_ans=dataframe_temp.iloc[i]['id']
                time=temp_time
                #print('candidate found at',time)
                # change the id of the candidate
                dataframe_temp.loc[dataframe_temp[dataframe_temp['id']==id_ans].index,'id']=id
                dataframe.loc[dataframe[dataframe['id']==id_ans].index,'id']=id
                find_fourmis(dataframe,min_time,max_time,id,time)
                #print('found at',time)
    #print('perdu a',time)
    return dataframe
    
        
        


In [33]:
len_list=dataframe['id'].unique()
new_dataframe=pd.DataFrame(columns=['id','center_x','center_y','time'])
for id in len_list:
    if id in dataframe['id'].unique():
        time= min(dataframe[dataframe['id']==id]['time'])
        dataframe=find_fourmis(dataframe,min_time,max_time,id,time)
        if len(dataframe[dataframe['id']==id])>300:
            # print('------------------------------')
            # print('id=',id)
            # print(dataframe[dataframe['id']==id][:5])
            # print(dataframe[dataframe['id']==id][-5:])
            # print('------------------------------')
            frames=[new_dataframe,dataframe[dataframe['id']==id]]
            new_dataframe=pd.concat(frames)

In [13]:
def renumerate(ids):
    result=np.zeros(len(ids),dtype=int)
    for i in range(len(ids)-1):
        if ids[i+1]>ids[i]:
            result[i+1:]+=1
    return(result)


In [34]:
ids=new_dataframe['id'].values
new_dataframe['id']=renumerate(ids)
new_dataframe

,id,center_x,center_y,time,x,y,w,h
17,0,502,1045,21,490.0,1039.0,24.0,12.0
18,0,502,1045,22,490.0,1039.0,24.0,12.0
19,0,496,1043,23,489.0,1037.0,14.0,13.0
20,0,495,1043,24,488.0,1037.0,14.0,13.0
21,0,500,1043,25,488.0,1037.0,25.0,13.0
...,...,...,...,...,...,...,...,...
125447,151,657,192,11854,650.0,180.0,14.0,25.0
125495,151,712,267,11855,702.0,263.0,21.0,9.0
125596,151,712,269,11856,701.0,262.0,22.0,14.0
126006,151,710,268,11859,700.0,262.0,21.0,13.0


In [35]:
new_dataframe=new_dataframe.drop_duplicates(subset=['time','id'],keep='last')
new_dataframe=new_dataframe.drop_duplicates(subset=['time','center_x','center_y'],keep='last')

In [36]:
fourmis_display=new_dataframe.sort_values(by=['time'])
fourmis_display.to_csv('result/boite/location_dectection_diplay.csv',index=False)

In [24]:
fourmis_display

,id,center_x,center_y,time,x,y,w,h
17,0,502,1045,21,490.0,1039.0,24.0,12.0
18,0,502,1045,22,490.0,1039.0,24.0,12.0
19,0,496,1043,23,489.0,1037.0,14.0,13.0
20,0,495,1043,24,488.0,1037.0,14.0,13.0
21,0,500,1043,25,488.0,1037.0,25.0,13.0
...,...,...,...,...,...,...,...,...
125831,150,968,788,11858,958.0,781.0,20.0,15.0
126006,148,710,268,11859,700.0,262.0,21.0,13.0
126006,151,710,268,11859,700.0,262.0,21.0,13.0
126100,148,712,268,11860,702.0,262.0,20.0,12.0
